## measurementTechnique extraction and standardization review

#### Analyze the extraction of measurementTechniques by ChatGPT
Approach:
- Select 10 records at random from each of the sampled repositories
- For each record, evaluate the techniques pulled by ChatGPT as True or False positive based on description
- Calculate precision and recall for each record
- If there are terms which are the same, but appear different, pull those out for standardization analysis

#### Analyse the extraction of extraneous generic 'stop word' terms
Approach:
- Split all ChatGPT generated terms into words (split by space)
- Generate frequency table of terms
- Identify top set of generic terms for use as permutations in standardization

#### Analyze the standardization of measurementTechnique terms
Approach:
- Create list of terms based based on mapping to multiple ontologies
- Create permutations in term list
- Run the standardization approach and calculate precision, recall F relative to original term

#### Determine threshold for cut off to favor false negatives over false positives
Approach:
- For each threshhold cutoff
  - Select 25 terms and determine how well they matched



In [2]:
import requests
import json
import os
import pandas as pd

In [3]:
script_path = os.getcwd()
data_path = os.path.join(script_path,'data')
result_path = os.path.join(script_path,'result')

### Analyze ChatGPT's extraction

In [ ]:
## Load the file
gpt_results = pd.read_csv(os.path.join(data_path,'GPT Measurement Techniques results.tsv'),delimiter='\t',header=0)
print(gpt_results.head(n=2))

## select 10 random records
ransamps = gpt_results.groupby('Data Repository').sample(10)

def ifenumed(rowdata):
    numlist = ['1.','2.','3.','4.','5.','6.','7.','8.','9.','10.','11.','12.','13.','14.','15.']
    for eachnum in numlist:
        if eachnum in rowdata:
            rowdata.replace(eachnum," - ")
        else:
            break
    return rowdata

## format results from records
def clean_predictions(row):
    rowdata = row['Predictions']
    if '1. ' in rowdata:
        rowdata = ifenumed(rowdata)
    tmpdata = rowdata.replace(" - ","|")
    tmplist = tmpdata.split("|")
    cleanlist = [x.replace("- ","").strip() for x in tmplist]
    return cleanlist 

ransamps['clean_pred'] = ransamps.apply(lambda row: clean_predictions(row), axis=1)
exploded_df = ransamps.explode('clean_pred')
#print(ransamps.head(n=2))
exploded_df.drop(columns=['Model','Predictions'],inplace=True)
print(exploded_df.head(n=2))

## Export results for manual evaluation
exploded_df.to_csv(os.path.join(result_path,'GPT_sample.tsv'),sep='\t',header=True)


### Analyse the extraction of extraneous generic 'stop word' terms

In [ ]:
## Pull the terms from Dylan's list (since the source is not needed)
t2t_results = pd.read_csv(os.path.join(data_path,'Measurement Techniques mapped.tsv'),delimiter='\t',header=2)

## Pull the list of techniques
techniques = t2t_results['Technique'].tolist()

## process the terms
cleanlist = []
for eachterm in techniques:
    tmpterm = eachterm.lower()
    tmpterm.replace("-"," ").replace(":"," ")
    tmplist = tmpterm.split(" ")
    cleantmp = [x.replace("(","").replace(")","").strip() for x in tmplist]
    cleanlist.extend(cleantmp)

termseries = pd.Series(cleanlist)
termdf = termseries.to_frame('technique')
termfreq = termdf.groupby('technique').size().reset_index(name='counts')
termfreq.sort_values('counts',ascending=False,inplace=True)
print(termfreq.loc[termfreq['counts']>5])
termfreq.to_csv(os.path.join(result_path,'stopword_freq.tsv'),sep='\t',header=0)

### Analyze the standardization of measurementTechnique terms

* mappings endpoint documentation: https://data.bioontology.org/documentation#Mapping
* sample code: https://gist.github.com/callahantiff/a28fb3160782f42f104e9ec41553af0d
* NCBO sample code: https://github.com/ncbo/ncbo_rest_sample_code

In [5]:
import urllib.request, urllib.error, urllib.parse

## Load the API key
with open(os.path.join(script_path,'config.json'),'rb') as keyfile:
    keyinfo = json.load(keyfile)
    apikey = keyinfo['apikey']

## Format the apikey for the header
def get_json(url, apikey):
    opener = urllib.request.build_opener()
    opener.addheaders = [('Authorization', 'apikey token=' + apikey)]
    return json.loads(opener.open(url).read())

## Provide the list of ontologies to map
onto_list = ["BAO","OBI","EFO","NCIT","EDAMT","MMO","CHMO"]

## Pull mapped pairs out of a paginated dictionary
def get_mappings(onto_source,page_dict):
    mappinglist = []
    for eachcollection in page_dict['collection']:
        tmpdict = {'source_ontology': onto_source,
                   'source_id': eachcollection['classes'][0]['@id'],
                   'map_method': eachcollection['source'],
                   'target_id': eachcollection['classes'][1]['@id']}
        mappinglist.append(tmpdict)
    return mappinglist

In [8]:
## filter the results to only mappings within ontologies of interest
def filter_for_ontos(onto_list,mappingdf):
    relevant_df = pd.DataFrame(columns=mappingdf.columns.tolist())
    source_ont = mappingdf.iloc[0]['source_ontology']
    target_list = [x for x in onto_list if x!=source_ont]
    for eachtarget in target_list:
        target_subset = mappingdf.loc[mappingdf['target_id'].str.contains(eachtarget)]
        relevant_df = pd.concat(([relevant_df,target_subset]),ignore_index=True)
    return relevant_df

def download_mappings(apikey,onto_list,starting_page):
    for each_onto in onto_list:
        print("now downloading mappings from: ",each_onto)
        allmappinglist = []
        ontologymap = f"https://data.bioontology.org/ontologies/{each_onto}/mappings"
        r = get_json(ontologymap,apikey)
        if starting_page == 0:
            page = get_json(r['links']["nextPage"],apikey)
        else:
            page = starting_page
        next_page = page
        while next_page:
            next_page = page["links"]["nextPage"]
            tmpmapping = get_mappings(each_onto, page)
            allmappinglist.extend(tmpmapping)
            if next_page:
                page = get_json(next_page,apikey)
                print(page["links"]["nextPage"])
        mappingdf = pd.DataFrame(allmappinglist)
        relevant_df = filter_for_ontos(onto_list,mappingdf)
        relevant_df.to_csv(os.path.join(result_path,"mappings",f"{each_onto}_mappings.tsv"), sep='\t', header=True)


In [9]:
%%time

## Download all mappings from ontologies of interest
download_mappings(apikey,onto_list,0)


now downloading mappings from:  BAO
https://data.bioontology.org/ontologies/BAO/mappings?page=4
https://data.bioontology.org/ontologies/BAO/mappings?page=5
https://data.bioontology.org/ontologies/BAO/mappings?page=6
https://data.bioontology.org/ontologies/BAO/mappings?page=7
https://data.bioontology.org/ontologies/BAO/mappings?page=8
https://data.bioontology.org/ontologies/BAO/mappings?page=9
https://data.bioontology.org/ontologies/BAO/mappings?page=10
https://data.bioontology.org/ontologies/BAO/mappings?page=11
https://data.bioontology.org/ontologies/BAO/mappings?page=12
https://data.bioontology.org/ontologies/BAO/mappings?page=13
https://data.bioontology.org/ontologies/BAO/mappings?page=14
https://data.bioontology.org/ontologies/BAO/mappings?page=15
https://data.bioontology.org/ontologies/BAO/mappings?page=16
https://data.bioontology.org/ontologies/BAO/mappings?page=17
https://data.bioontology.org/ontologies/BAO/mappings?page=18
https://data.bioontology.org/ontologies/BAO/mappings?pa

https://data.bioontology.org/ontologies/BAO/mappings?page=138
https://data.bioontology.org/ontologies/BAO/mappings?page=139
https://data.bioontology.org/ontologies/BAO/mappings?page=140
https://data.bioontology.org/ontologies/BAO/mappings?page=141
https://data.bioontology.org/ontologies/BAO/mappings?page=142
https://data.bioontology.org/ontologies/BAO/mappings?page=143
https://data.bioontology.org/ontologies/BAO/mappings?page=144
https://data.bioontology.org/ontologies/BAO/mappings?page=145
https://data.bioontology.org/ontologies/BAO/mappings?page=146
https://data.bioontology.org/ontologies/BAO/mappings?page=147
https://data.bioontology.org/ontologies/BAO/mappings?page=148
https://data.bioontology.org/ontologies/BAO/mappings?page=149
https://data.bioontology.org/ontologies/BAO/mappings?page=150
https://data.bioontology.org/ontologies/BAO/mappings?page=151
https://data.bioontology.org/ontologies/BAO/mappings?page=152
https://data.bioontology.org/ontologies/BAO/mappings?page=153
https://

https://data.bioontology.org/ontologies/BAO/mappings?page=271
https://data.bioontology.org/ontologies/BAO/mappings?page=272
https://data.bioontology.org/ontologies/BAO/mappings?page=273
https://data.bioontology.org/ontologies/BAO/mappings?page=274
https://data.bioontology.org/ontologies/BAO/mappings?page=275
https://data.bioontology.org/ontologies/BAO/mappings?page=276
https://data.bioontology.org/ontologies/BAO/mappings?page=277
https://data.bioontology.org/ontologies/BAO/mappings?page=278
https://data.bioontology.org/ontologies/BAO/mappings?page=279
https://data.bioontology.org/ontologies/BAO/mappings?page=280
https://data.bioontology.org/ontologies/BAO/mappings?page=281
https://data.bioontology.org/ontologies/BAO/mappings?page=282
https://data.bioontology.org/ontologies/BAO/mappings?page=283
https://data.bioontology.org/ontologies/BAO/mappings?page=284
https://data.bioontology.org/ontologies/BAO/mappings?page=285
https://data.bioontology.org/ontologies/BAO/mappings?page=286
https://

https://data.bioontology.org/ontologies/BAO/mappings?page=404
https://data.bioontology.org/ontologies/BAO/mappings?page=405
https://data.bioontology.org/ontologies/BAO/mappings?page=406
https://data.bioontology.org/ontologies/BAO/mappings?page=407
https://data.bioontology.org/ontologies/BAO/mappings?page=408
https://data.bioontology.org/ontologies/BAO/mappings?page=409
https://data.bioontology.org/ontologies/BAO/mappings?page=410
https://data.bioontology.org/ontologies/BAO/mappings?page=411
https://data.bioontology.org/ontologies/BAO/mappings?page=412
https://data.bioontology.org/ontologies/BAO/mappings?page=413
https://data.bioontology.org/ontologies/BAO/mappings?page=414
https://data.bioontology.org/ontologies/BAO/mappings?page=415
https://data.bioontology.org/ontologies/BAO/mappings?page=416
https://data.bioontology.org/ontologies/BAO/mappings?page=417
https://data.bioontology.org/ontologies/BAO/mappings?page=418
https://data.bioontology.org/ontologies/BAO/mappings?page=419
https://

https://data.bioontology.org/ontologies/BAO/mappings?page=537
https://data.bioontology.org/ontologies/BAO/mappings?page=538
https://data.bioontology.org/ontologies/BAO/mappings?page=539
https://data.bioontology.org/ontologies/BAO/mappings?page=540
https://data.bioontology.org/ontologies/BAO/mappings?page=541
https://data.bioontology.org/ontologies/BAO/mappings?page=542
https://data.bioontology.org/ontologies/BAO/mappings?page=543
https://data.bioontology.org/ontologies/BAO/mappings?page=544
https://data.bioontology.org/ontologies/BAO/mappings?page=545
https://data.bioontology.org/ontologies/BAO/mappings?page=546
https://data.bioontology.org/ontologies/BAO/mappings?page=547
https://data.bioontology.org/ontologies/BAO/mappings?page=548
https://data.bioontology.org/ontologies/BAO/mappings?page=549
https://data.bioontology.org/ontologies/BAO/mappings?page=550
https://data.bioontology.org/ontologies/BAO/mappings?page=551
https://data.bioontology.org/ontologies/BAO/mappings?page=552
https://

...

https://data.bioontology.org/ontologies/NCIT/mappings?page=733
https://data.bioontology.org/ontologies/NCIT/mappings?page=734
https://data.bioontology.org/ontologies/NCIT/mappings?page=735
https://data.bioontology.org/ontologies/NCIT/mappings?page=736
https://data.bioontology.org/ontologies/NCIT/mappings?page=737
https://data.bioontology.org/ontologies/NCIT/mappings?page=738
https://data.bioontology.org/ontologies/NCIT/mappings?page=739
https://data.bioontology.org/ontologies/NCIT/mappings?page=740
https://data.bioontology.org/ontologies/NCIT/mappings?page=741
https://data.bioontology.org/ontologies/NCIT/mappings?page=742
https://data.bioontology.org/ontologies/NCIT/mappings?page=743
https://data.bioontology.org/ontologies/NCIT/mappings?page=744
https://data.bioontology.org/ontologies/NCIT/mappings?page=745
https://data.bioontology.org/ontologies/NCIT/mappings?page=746
https://data.bioontology.org/ontologies/NCIT/mappings?page=747
https://data.bioontology.org/ontologies/NCIT/mappings?p

https://data.bioontology.org/ontologies/NCIT/mappings?page=864
https://data.bioontology.org/ontologies/NCIT/mappings?page=865
https://data.bioontology.org/ontologies/NCIT/mappings?page=866
https://data.bioontology.org/ontologies/NCIT/mappings?page=867
https://data.bioontology.org/ontologies/NCIT/mappings?page=868
https://data.bioontology.org/ontologies/NCIT/mappings?page=869
https://data.bioontology.org/ontologies/NCIT/mappings?page=870
https://data.bioontology.org/ontologies/NCIT/mappings?page=871
https://data.bioontology.org/ontologies/NCIT/mappings?page=872
https://data.bioontology.org/ontologies/NCIT/mappings?page=873
https://data.bioontology.org/ontologies/NCIT/mappings?page=874
https://data.bioontology.org/ontologies/NCIT/mappings?page=875
https://data.bioontology.org/ontologies/NCIT/mappings?page=876
https://data.bioontology.org/ontologies/NCIT/mappings?page=877
https://data.bioontology.org/ontologies/NCIT/mappings?page=878
https://data.bioontology.org/ontologies/NCIT/mappings?p

https://data.bioontology.org/ontologies/NCIT/mappings?page=995
https://data.bioontology.org/ontologies/NCIT/mappings?page=996
https://data.bioontology.org/ontologies/NCIT/mappings?page=997
https://data.bioontology.org/ontologies/NCIT/mappings?page=998
https://data.bioontology.org/ontologies/NCIT/mappings?page=999
https://data.bioontology.org/ontologies/NCIT/mappings?page=1000
https://data.bioontology.org/ontologies/NCIT/mappings?page=1001
https://data.bioontology.org/ontologies/NCIT/mappings?page=1002
https://data.bioontology.org/ontologies/NCIT/mappings?page=1003
https://data.bioontology.org/ontologies/NCIT/mappings?page=1004
https://data.bioontology.org/ontologies/NCIT/mappings?page=1005
https://data.bioontology.org/ontologies/NCIT/mappings?page=1006
https://data.bioontology.org/ontologies/NCIT/mappings?page=1007
https://data.bioontology.org/ontologies/NCIT/mappings?page=1008
https://data.bioontology.org/ontologies/NCIT/mappings?page=1009
https://data.bioontology.org/ontologies/NCIT/

https://data.bioontology.org/ontologies/NCIT/mappings?page=1124
https://data.bioontology.org/ontologies/NCIT/mappings?page=1125
https://data.bioontology.org/ontologies/NCIT/mappings?page=1126
https://data.bioontology.org/ontologies/NCIT/mappings?page=1127
https://data.bioontology.org/ontologies/NCIT/mappings?page=1128
https://data.bioontology.org/ontologies/NCIT/mappings?page=1129
https://data.bioontology.org/ontologies/NCIT/mappings?page=1130
https://data.bioontology.org/ontologies/NCIT/mappings?page=1131
https://data.bioontology.org/ontologies/NCIT/mappings?page=1132
https://data.bioontology.org/ontologies/NCIT/mappings?page=1133
https://data.bioontology.org/ontologies/NCIT/mappings?page=1134
https://data.bioontology.org/ontologies/NCIT/mappings?page=1135
https://data.bioontology.org/ontologies/NCIT/mappings?page=1136
https://data.bioontology.org/ontologies/NCIT/mappings?page=1137
https://data.bioontology.org/ontologies/NCIT/mappings?page=1138
https://data.bioontology.org/ontologies/

HTTPError: HTTP Error 502: Bad Gateway

In [11]:
## Filter terms to children of relevant branches in the ontology tree


NameError: name 'allmappinglist' is not defined

In [ ]:
## Pull synonymous terms based on the mappings

In [ ]:
## mutate the terms

### Figuring out the data structure of a result from the API

The organization of the mappings appear to be as follows:
A result from the API ontology/mappings endpoint:
* r.keys:  dict_keys(['page', 'pageCount', 'totalCount', 'prevPage', 'nextPage', 'links', 'collection'])
  * r['collection'].keys():  dict_keys(['id', 'source', 'classes', 'process', '@id', '@type'])
    * r['collection'][0]['classes'].keys:  dict_keys(['@id', '@type', 'links', '@context'])

Where each pair of mapped terms appear to be listed under 'classes' with classes[0] being the term in the source ontology and classes[1] being the term from a different ontology

The source is how two terms were mapped

In [ ]:
## Test to see that an API request is working
REST_URL = "http://data.bioontology.org"
term = "survey"
r = get_json(REST_URL + "/search?q=" + term,apikey)["collection"]
#print(r)

In [ ]:
## Testing the class mapping endpoint
ontology_shorthand = 'BRO'
classurl = 'http%3A%2F%2Fbioontology.org%2Fontologies%2FBiomedicalResourceOntology.owl%23Ontology_Development_and_Management'
classmap = f"https://data.bioontology.org/ontologies/{ontology_shorthand}/classes/{classurl}/mappings"
r = get_json(classmap,apikey)
print('r[0].keys: ', r[0].keys())
print('r[0][classes][0].keys: ', r[0]['classes'][0].keys())
print('r[0][classes][0][links]: ', r[0]['classes'][0]['links'].keys())
print(r[0]['classes'][0]['links']['descendants'])

In [ ]:
## Test the ontology mapping end point
ontology_shorthand = 'MMO'
classurl = 'http://purl.obolibrary.org/obo/MMO_0000000'
ontologymap = f"https://data.bioontology.org/ontologies/{ontology_shorthand}/mappings"
r = get_json(ontologymap,apikey)

print("r.keys: ",r.keys())
print("r['links']: ", r['links'])
print("r['page']: ", r['page'])
print("r['collection'].keys(): ", r['collection'][0].keys())
print("r['collection'][0]['classes'].keys: ", r['collection'][0]['classes'][0].keys())
print("collection id: ",r['collection'][0]["@id"])
print("class id: ", r['collection'][0]['classes'][1]["@id"])
print("class type: ", r['collection'][0]['classes'][1]["@type"])
print("class context: ", r['collection'][0]['classes'][1]["@context"])
print("number of classes: ",len(r['collection'][0]['classes']))

print(r['pageCount'])
for eachcollection in r['collection'][0:3]:
    #print(len(eachcollection['classes']))
    print(eachcollection['classes'][0]['@id'],eachcollection['source'],eachcollection['classes'][1]['@id'])
    print(eachcollection['classes'][0]['@context'],eachcollection['classes'][1]['@context'])

In [ ]:
## Test the use of pagination
page = get_json(r['links']["nextPage"],apikey)
allmappinglist = []
# Iterate over the available pages adding labels from all classes
# When we hit the last page, the while loop will exit
next_page = page
while next_page:
    next_page = page["links"]["nextPage"]
    tmpmapping = get_mappings("MMO", page)
    allmappinglist.extend(tmpmapping)
    if next_page:
        page = get_json(next_page,apikey)

In [ ]:
mappingdf = pd.DataFrame(mappinglist)
print(mappingdf.head(n=2))

In [ ]:
print(len(mappingdf))